In [21]:
# Add path with autoencoding code
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../code_autoencoding')

#Torch and utilities
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable


import numpy as np
import pandas as pd
import glob
import re

from cmapPy.pandasGEXpress.parse import parse
import data_loader as dl
import autoencoder
import trainer

In [34]:
data_path = "../data/HF_LM_genes_counts.csv"
pEF_df = pd.read_csv(data_path)

vec_keys = [k for k in pEF_df.columns if re.search('X\d{3,5}', k)]

pEF_vectors = dl.vectorize(pEF_df.iloc[:, pEF_df.columns.isin(vec_keys)], scale_log2_1 = True )

#pEF_vectors = {k : np.array(pEF_df[k]) for k in vec_keys}


In [35]:
pEF_vectors_torch = dl.TorchVectors(pEF_vectors)
eval_loader = DataLoader(pEF_vectors_torch, batch_size=1, pin_memory=True, shuffle=True)

In [36]:
model_param_files = glob.glob("../trained_model_parameters/leakyReLu/3.35e-06_withHF_counts150epoch_leakyReLu_962_1024_model_best.pth")
#model_param_files = ['../trained_model_parameters/leakyReLu/1.36e-06_counts150epoch_leakyReLu_960_1100_model_best.pth']
criterion = nn.MSELoss()

for f in model_param_files:
    
    # Load parameters from training
    params = torch.load(f)
    
    # Get the dimensions of the first layer and the weights
    latent_size, input_size = params['state_dict']['net.0.weight'].shape
    
    AE = autoencoder.Autoencoder(input_size,latent_size)
    AE.set_weights(params)
    #AE.double()
    AE.cuda()
    
    loss = trainer.val_step(AE, eval_loader, prt = False)
    print(f)
    print(loss)
    


../trained_model_parameters/leakyReLu/3.35e-06_withHF_counts150epoch_leakyReLu_962_1024_model_best.pth
0.0012213991544938283


In [16]:
latent_size

1100